<a href="https://colab.research.google.com/github/robinrb7/TransferLearning_FeatureExtraction/blob/main/Tranfer_Learning__Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!git clone https://github.com/robinrb7/TransferLearning_FeatureExtraction.git

Cloning into 'TransferLearning_FeatureExtraction'...


In [ ]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

--2025-01-07 17:35:59--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.128.207, 74.125.143.207, 173.194.69.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.128.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M  37.9MB/s    in 4.9s    

2025-01-07 17:36:04 (33.0 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [ ]:
import os

for dirpath,dirnames,filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories in {len(filenames)} images in '{dirpath}'. ")

There are 2 directories in 0 images in '10_food_classes_10_percent'. 
There are 10 directories in 0 images in '10_food_classes_10_percent/train'. 
There are 0 directories in 75 images in '10_food_classes_10_percent/train/grilled_salmon'. 
There are 0 directories in 75 images in '10_food_classes_10_percent/train/chicken_wings'. 
There are 0 directories in 75 images in '10_food_classes_10_percent/train/chicken_curry'. 
There are 0 directories in 75 images in '10_food_classes_10_percent/train/fried_rice'. 
There are 0 directories in 75 images in '10_food_classes_10_percent/train/steak'. 
There are 0 directories in 75 images in '10_food_classes_10_percent/train/ice_cream'. 
There are 0 directories in 75 images in '10_food_classes_10_percent/train/sushi'. 
There are 0 directories in 75 images in '10_food_classes_10_percent/train/hamburger'. 
There are 0 directories in 75 images in '10_food_classes_10_percent/train/pizza'. 
There are 0 directories in 75 images in '10_food_classes_10_percent/

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_gen = ImageDataGenerator(rescale= 1/255.)
test_gen = ImageDataGenerator(rescale= 1/255.)

train_data = train_gen.flow_from_directory(train_dir,
                                           target_size=(224,224),
                                           batch_size = 32,
                                           class_mode="categorical"
                                           )
test_data = test_gen.flow_from_directory(test_dir,
                                         target_size=(224,224),
                                         batch_size=32,
                                         class_mode="categorical")


Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [ ]:
import datetime
import tensorflow as tf

def create_tensorboard_callback(dirname,exp_name):
  log_dir = dirname + "/" + exp_name + "/" + datetime.datetime.now().strftime("%Y%m%d - %H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
  print(f"Saving tensor board log files to : {log_dir} ")
  return tensorboard_callback

In [ ]:
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [ ]:
!pip install tensorflow==2.15.0 tensorflow-hub keras==2.15.0

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.0
    Uninstalling wrapt-1.17.0:
      Successfully uninstalled wrapt-1.17.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
def create_model(model_url, num_classes=10):

  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False, # freeze the already learned patterns
                                           name="feature_extraction_lyaer",
                                           input_shape=(224,224,3))

  # Create our own model
  model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(num_classes, activation="softmax", name="output_layer")
  ])

  return model

In [ ]:
resnet_model = create_model(resnet_url,
                            num_classes=train_data.num_classes)

In [ ]:
resnet_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [ ]:
resnet_history = resnet_model.fit(train_data,
                                  epochs=5,
                                  steps_per_epoch=len(train_data),
                                  validation_data=test_data,
                                  validation_steps=len(test_data),
                                  callbacks=[create_tensorboard_callback(dirname="tensorflow_hub",
                                                                         exp_name="resnet50V2"
                                                                         )])

Saving tensor board log files to : tensorflow_hub/resnet50V2/20250107 - 174323 
Epoch 1/5
24/24 [==============================] - 22s 614ms/step - loss: 1.9100 - accuracy: 0.3533 - val_loss: 1.1917 - val_accuracy: 0.6200
Epoch 2/5
24/24 [==============================] - 11s 477ms/step - loss: 0.9092 - accuracy: 0.7147 - val_loss: 0.8714 - val_accuracy: 0.7140
Epoch 3/5
24/24 [==============================] - 11s 474ms/step - loss: 0.6359 - accuracy: 0.8080 - val_loss: 0.7740 - val_accuracy: 0.7448
Epoch 4/5
24/24 [==============================] - 12s 501ms/step - loss: 0.4863 - accuracy: 0.8653 - val_loss: 0.7165 - val_accuracy: 0.7652
Epoch 5/5
24/24 [==============================] - 13s 553ms/step - loss: 0.3927 - accuracy: 0.9120 - val_loss: 0.6753 - val_accuracy: 0.7784


In [ ]:
import matplotlib.pyplot as plt


def plot_loss_curves(history):

  loss = history.history["loss"]
  val_loss = history.history["val_loss"]

  accuracy = history.history["accuracy"]
  val_accuracy = history.history["val_accuracy"]

  epochs = range(len(history.history["loss"]))

  # Plot loss
  plt.plot(epochs, loss, label="training_loss")
  plt.plot(epochs, val_loss, label="val_loss")
  plt.title("Loss")
  plt.xlabel("Epochs")
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label="training_accuracy")
  plt.plot(epochs, val_accuracy, label="val_accuracy")
  plt.title("Accuracy")
  plt.xlabel("Epochs")
  plt.legend();

In [ ]:
plot_loss_curves(resnet_history)

In [ ]:
efficientnet_model = create_model(efficientnet_url,
                            num_classes=train_data.num_classes)

efficientnet_model.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

efficientnet_history = efficientnet_model.fit(train_data,
                                  epochs=5,
                                  steps_per_epoch=len(train_data),
                                  validation_data=test_data,
                                  validation_steps=len(test_data),
                                  callbacks=[create_tensorboard_callback(dirname="tensorflow_hub",
                                                                         exp_name="efficientnetb0"
                                                                         )])

In [ ]:
plot_loss_curves(efficientnet_history)

In [ ]:
!tensorboard dev upload --logdir ./tensorflow_hub/ \
  --name "EfficientNetB0 vs ResNet50v2" \
  --description "Comparing two different Feature extraction model "\
  --one_shot

2025-01-07 17:48:15.262790: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-07 17:48:15.262839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-07 17:48:15.264129: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 17:48:16.245760: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
****************************************************************
****************************************************************
****************************************************************

Uploading TensorBoard logs to https://tensorb